<div dir="rtl">
در این قسمت کلاس 
activation 
را میسازیم که همه تابع های فعال ساز و متود مشتق آن هارا در این کلاس قرار می دهیم
و در ابتدا تابع سازنده این کلاس را می نویسیمو در ادامه ازین کلاس در هر نود که ساختیم ابجکت میسازسم و بسته به این که تابع فعال ساز آن لایه را چه وارد کرده ایم متود فعال ساز آن لایه را فراخوانی می کنیم از این کلاس استفاده می کنیم . 
</div>

In [1]:
import math
class Activation:
   # with out use np
    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))
    def sigmoid_prime(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    def relu(self, x):
        return max(0.1 * x, x)
    def relu_prime(self, x):
        return 1 if x > 0 else 0.1
    def step(self, x):
        return 1 if x > 0 else 0
    def step_prime(self, x):
        return  1
    
    def relu_prime(self, x):
        return 1 if x > 0 else 0
    def tanh(self, x):
        return math.tanh(x)
    def tanh_prime(self, x):
        return 1 - math.tanh(x) ** 2
    def __call__(self, x):
        return self.sigmoid(x)
    def prime(self, x):
        return self.sigmoid_prime(x)

<div dir="rtl">
در این قسمت کلاس نود را پیاده سازی می کنیم که این نود ها همان نود های شبکه عصبی هستند که در قسمت 
init
مقدار های اولیه آن ها را می دهیم و 
weight , bias
را که در اصل همان وزن هایی است که این نود با ضرب شدن و در نهایت جمع شدن در آن به وجود می آید و 
output
همان خروجی نهایی هر نورون پس از عبور از فعال ساز است و 
input
ورودوی این نورون یا همان خروجی لایه قبل است و 
nextLweights
هم وزن های لایه بعد است که از این نورون به نود های لایه بعدی وصل شده است 
و
nextdeltas
هم ذلتا های لایه بعد است که برای پیدا کردن دلتای  این لایه به آن ها نیاز داریم
و
biasprime , weightprime
هم مشتق وزن ها و بایاس این نورون است که برای اپدیت
به این ها نیاز داریم
و
Activatioin
که کلاس است را در هر نود ابتدا تشکیل می دهیم که از تابع های آن استفاده کنیم
و
z
هم خروجی قبل از رد شدن از تابع فعال ساز است 
و
activation
تابع این نورون است که تعیین می کنیم که چه تابعی باشد و همچنین 
activationprime
هم با تئجه به این که گفتع شده چه تابعی فعال ساز باشد مشتق آن هم مشخص می کنیم.
تابع 
forward
ورودی این نورون که مشخص شده است 
و ورودی را در بردار وزن ضرب داخلی می کنیم که برابر
z
است و
از آن تابع فعال ساز که مشخص کردیم میگیریم و 
output 
 را به ما می دهد.
و تابع های get
را می نویسیم که اگر خواستیم دریافت کنیم پارامتر های مورد نظر را بتوانیم.
update
به این صورت عمل می کند که مقدار وزن یا بایاس مورد نظر را منهای مشتق ان ضرب در 
learning rate 
می کنیم .
قسمت 
compute delta
از فرمول حساب دلتا و با داشتن 
وزن ها و دلتای بعدی می توانیم دلتای این نورون را محاسبه کنیم.
و محاسبه دلتا برای لایه اخر متفاوت است برای همین دو حالت می نویسیم.
در 
make_weights_bias_prime
مشتق نسبت به وزن ها و بایاس را باداشتن دلتا و ورودی این نورون که همان خروجی لایه قبل است محاسبه می کنیم.
و بقیه تابع ها 
set 
هستند که برای ست کردن پارامتر ها استفاده میکنیم.

</div>

In [2]:
import numpy as np
class Node:
    # with random initialization 
    def __init__(self, lastlayer_size, nextlayer_size, activation):
        #random initialization between 0 and 5
        self.weights = np.random.rand(lastlayer_size)
        self.bias = np.random.rand()
        self.output = 0
        self.delta = 0
        self.input = [0] * lastlayer_size
        self.nextLweights = [0] * nextlayer_size
        self.nextdeltas = [0] * nextlayer_size
        self.biasprime = 0
        self.weightsprime = [0] * lastlayer_size
        self.activation = activation
        self.z = 0
        self.Activation_functions = Activation()


        if self.activation == "sigmoid":
            self.activation_function = self.Activation_functions.sigmoid
        elif self.activation == "relu":
            self.activation_function = self.Activation_functions.relu
        elif self.activation == "tanh":
            self.activation_function = self.Activation_functions.tanh
        elif self.activation == "step":
            self.activation_function = self.Activation_functions.step
        else:
            self.activation_function = self.Activation_functions.sigmoid
        

        if self.activation == "sigmoid":
            self.activation_function_prime = self.Activation_functions.sigmoid_prime
        elif self.activation == "relu":
            self.activation_function_prime = self.Activation_functions.relu_prime
        elif self.activation == "tanh":
            self.activation_function_prime = self.Activation_functions.tanh_prime
        elif self.activation == "step":
            self.activation_function_prime =self.Activation_functions.step_prime
        else:
            self.activation_function_prime = self.Activation_functions.sigmoid_prime


        # make output in this part
    def forward(self, x,isoutputlayer = False ):


        self.input = x
        self.z = sum([i * j for i, j in zip(self.weights, x)]) + self.bias
        if isoutputlayer :
            self.output =self.activation_function(self.z)
        else:
            self.output = self.activation_function(self.z)
        return self.output

    def getz(self):
        return self.z

    def getdelta(self):
        return self.delta
    
    def update(self, lr):
        self.weights = [i - lr * j for i, j in zip(self.weights, self.weightsprime)]
        self.bias = self.bias - lr * self.biasprime


        # print the weightsprime
        return self.biasprime
    
    def compute_delta(self, real_y,isoutputlayer = False ):
        # nextweights dot nextdeltas multiply by sigmoid prime
     
        if isoutputlayer:
            # mse prime
            self.delta = 2*(self.output - real_y) * self.activation_function_prime(self.z)
        else:
            self.delta = sum([i * j for i, j in zip(self.nextLweights, self.nextdeltas)]) * self.activation_function_prime(self.z)
          
            
    
    def make_weights_bias_prime(self):
        self.weightsprime = [i * self.delta for i in self.input]
        self.biasprime = self.delta
     
    def setInputs(self, inputs):
        self.input = inputs
    
    def setNextLweights(self, weights):
        self.nextLweights = weights
    
    def setNextLdeltas(self, deltas):
        self.nextdeltas = deltas
    
    def getweights_prime(self):
        return self.weightsprime
    def getbias_prime(self):
        return self.biasprime
    

    def setweights_prime(self,weights_prime):
        self.weightsprime = weights_prime
    def setbias_prime(self,bias_prime):
        self.biasprime = bias_prime
        
        
        
        

<div dir="rtl">
در این قسمت کلاس لایه را پیاده سازی می کنیم که این کلاس در قسمت مقدار دهی اولیه
یک لیست از نود ها دارد که که همان نود های است مه در آن لایه قرار دارند و ساز این لایه که تعداد نورون های این لایه است را ورودی گرفته ایم و 
در ابتدا به همان تعداد ابجکت نود میسازیم 
و 
outputs
 که همان تمام خروجی های نود های این لایه است را در این لیست ذخیره میکنیم 
 deltas
 هم تمام دلتا ها این نود هاست که در لیست نگه می داریم
 و
 forward
 آن به این صورت است که تمام نود های این لایه را 
 forward 
  میکنیم .
  و
  setinputs
  هم ورودی تمام نود هارا برابر
  x
  قرار می دهیم که خروجی لایه قبل است.
  و
  getdeltas
  هم تمام دلتا های نود های این لایه را در یم لیست به ما بر میگرداند.
  getweights_related_i
  این تابع می خواهد به ازای تمام نود ها وزن 
  i
  ام آن را در یک لیست برگرداند چون
  چون برای لایه قبل برای پیدا کردن وزن هایی که به یک نورون خاص در لایه قبل وصل هستند یا همان
  nextweights
  لایه قبل به آن نیاز داریم.
  compute_deltas
  ,
  update_weights
  و
  compute_weights_bias_prime
  هم برای هر نورون همین تابع را در هر نورون صدا می زند.
  set_nextLweights_and_deltas
  همانطور که گفتیم با استفاده از تابع 
  getweights_related_i
  برای هر نورون با داشتن ابجکت لایه بعد می توانیم وزن های لایه بعد و دلتای لایه بعد را برای هر نورون محاسبه کنیم..
  set_weights_prime
  و
  set_bias_prime
  هم برای هر نورون این تابع را صدا میزند.

  bacward
  در لایه اول 
  از
  compute_deltas
  شروع می کنیم و برای هر نورون این تابع را صدا میزنیم و سپس
  compute_weights_bias_prime
  را صدا میزنیم و برای هر نورون این تابع را صدا میزنیم و مشتق نسبت به وزن ها و بایاس هارا در ایان لایه پیدا می کنیم.
  
</div>

In [3]:
class Layer:
    
    def __init__(self,size,lastlayer_size,nextlayer_size,activation):
        self.nodes = [Node(lastlayer_size,nextlayer_size,activation) for i in range(size)]
        self.ouuputs = []
        self.deltas = []
    def forward(self,x,isoutputlayer = False):
        self.outputs = [node.forward(x,isoutputlayer) for node in self.nodes]
        return self.outputs
    def set_inputs(self,x,isoutputlayer = False ):
        for i in range(len(self.nodes)):
            self.nodes[i].set_input(x)

    def getdeltas(self):
        self.deltas = [node.getdelta() for node in self.nodes]
        return self.deltas
    
    def getweights_related_i(self,i):
        return [node.weights[i] for node in self.nodes]
    

    def compute_deltas(self,real_y = 0,isoutputlayer = False):
        for i in range(len(self.nodes)):
            self.nodes[i].compute_delta(real_y ,isoutputlayer)

    def compute_weights_bias_prime(self):
        for i in range(len(self.nodes)):
            self.nodes[i].make_weights_bias_prime()

    def update_weights(self,lr):
        weightsprime = []
        for i in range(len(self.nodes)):
            weightsprime.append(self.nodes[i].update(lr))
        # print("weightsprime",weightsprime)

    def set_nextLweights_and_deltas(self,nextlayer):
        deltas = nextlayer.getdeltas()
        for i in range(len(self.nodes)):
            self.nodes[i].setNextLweights(nextlayer.getweights_related_i(i))
            self.nodes[i].setNextLdeltas(deltas)
           
    
    def backward(self,real_y = 0,isoutputlayer = False):
        self.compute_deltas(real_y ,isoutputlayer)
        self.compute_weights_bias_prime()
        # self.update_weights(lr=1)


        
    def getweights_prime(self):
        weightsprime = []
        for i in range(len(self.nodes)):
            weightsprime.append(self.nodes[i].getweights_prime())
        return weightsprime
    def getbias_prime(self):
        biasprime = []
        for i in range(len(self.nodes)):
            biasprime.append(self.nodes[i].getbias_prime())
        return biasprime
    def set_weights_prime(self,weights_prime):
        for i in range(len(self.nodes)):
            self.nodes[i].setweights_prime(weights_prime[i])
    def set_bias_prime(self,bias_prime):
        for i in range(len(self.nodes)):
            self.nodes[i].setbias_prime(bias_prime[i])
    
        

<div dir="rtl">
حال در این قسمت کلاس 
Mlp
را پیاده سازی می کنیم که در ابتدا لیستی از ابجکت های لایه هاست که به تعداد لایه ها لایه می سازیم و تعداد نورون لایه قبل و بعد را به آن میدهیم و همچنین اسم تابع فعال ساز مخصوص این لایه را.
همچنین یک لایه با یک نورون برای لایه نهاییکه مخفی نیست هم می سازیم.
ورودی را هم به لایه مخفی اول یا اگر لایه مخفی نداشت به لایه اخر می دهیم.
تابع 
forward
هم به این صورت است که تمام لایه ها را
forward
می کنیم.
و
خروجی هر لایه را به عنوان ورودی لایه بعدی می دهد.
و
call
همان 
forward
را صدا می زند.
و تابع 
parametes
هم تمام وزن ها و بایاس ها را بر می گرداند.
تابع
backward
ابتدا 
bacward
را از لایه اخر بر روی این لایه میزنذ تا م شتق نسبت به لایه را پیدا کندو با استفاده از تابع 
set_nextLweights_and_deltas
وزن ها و دلتای این لایه مورد نظر که در 
for
روی آن هستیم را به لایه بعد منتقل می کنیم.
و
loss
که همان تابع خطا است را محاسبه می کنیم.
با استفاده از فرمول مورد نظر و  گرفتن وردی واقعی.
set_weightsLi
و
set_biasLi
هم برای هر لایه این تابع را صدا میزنیم.
و
weight , bias
لایه مورد نظر که با شمار هان مشخص شده به هر نورون آن وارد می کنیم.
و در ادامه تابع های 
get 
موارد مورد نظر را از تمام لایه ها گرفته و در یک لیست قرار می دهد و به ما بر میگرداند.
و
set
ها هم همین صورت این چنین لیستی را گرفته و برای همه لای ها مقادیر مرود نظر را 
set 
 می کند.
 update
    هم برای هر لایه این تابع را صدا میزنیم.و 
learning rate
را منتقل می کنیم.

</div>

In [4]:
class Mlp :
    def __init__(self,layer_sizes,input_size,activations):
        if len(layer_sizes) == 1:
            self.layers = [Layer(layer_sizes[0],input_size,1,activations[0])]
            self.layers.append(Layer(1,layer_sizes[-1],0,activations[-2]))
        elif len(layer_sizes) == 0:
            self.layers = []
            self.layers.append(Layer(1,input_size,0,activations[-1]))
        else :
            self.layers = [Layer(layer_sizes[0],input_size,layer_sizes[0],activations[0])]
            for i in range(1,len(layer_sizes)-1):
                self.layers.append(Layer(layer_sizes[i],layer_sizes[i-1],layer_sizes[i+1],activations[i]))
            self.layers.append(Layer(layer_sizes[-1],layer_sizes[-2],1,activations[-2]))
            self.layers.append(Layer(1,layer_sizes[-1],0,activations[-1]))
        self.input = [0] * input_size

    def forward(self):
        x = self.input
        for i in range(len(self.layers)-1):
            x = self.layers[i].forward(x)
        x = self.layers[len(self.layers)-1].forward(x,True)
        return x[0]
       
    def __call__(self):
        self.forward()
        pass
    
    def parameters(self):
        weights = []
        for i in range(len(self.layers)):
            for j in range(len(self.layers[i].nodes)):
                weights.append(self.layers[i].nodes[j].weights)
        return weights
    def set_input(self,x):
        self.input = x

    
    def Backward(self,real_y,lr = 1):
        if len(self.layers) == 1:
            self.layers[0].backward(real_y,True)
          
        else:
            self.layers[-1].backward(real_y,True)
            self.layers[-2].set_nextLweights_and_deltas(self.layers[-1])
            for i in range(len(self.layers)-2,0,-1):
                self.layers[i].backward()
                self.layers[i-1].set_nextLweights_and_deltas(self.layers[i])
            self.layers[0].backward()
            return self.getweights_prime() , self.getbias_prime()

    def loss(self,real_y):
        return (self.forward() - real_y)**2
    
    def set_weightsLi(self,layernum,weights):
        for i in range(len(weights)):
            self.layers[layernum].nodes[i].weights = weights[i]
    def set_biasLi(self,layernum,bias):
        for i in range(len(bias)):
            self.layers[layernum].nodes[i].bias = bias[i]


    def getweights_prime(self):
        weights = []
        for i in range(len(self.layers)):
            weights.append(self.layers[i].getweights_prime())
        return weights
    
    def getbias_prime(self):
        bias = []
        for i in range(len(self.layers)):
            bias.append(self.layers[i].getbias_prime())
        return bias
    def set_weights_prime(self,weights):
        for i in range(len(self.layers)):
            self.layers[i].set_weights_prime(weights[i])
    def set_bias_prime(self,bias):
        for i in range(len(self.layers)):
            self.layers[i].set_bias_prime(bias[i])
    def update_weights_and_bias(self,lr):
        for i in range(len(self.layers)):
            self.layers[i].update_weights(lr)
            

<div dir="rtl">
کلاس
optimizer
را میسازیم که این کلاس پارامتر هارا می گیردو خود 
model
را
و در قسمت
step 
مقدار های وزن ها و بایاس ها را با استفاده از تابع
update
که در کلاس
Mlp
پیاده سازی شده است اپدیت می کند.
</div>

In [5]:
class Optimizer:
    def __init__(self,lr,model: Mlp, parameters = None):
    # Initialize the optimizer with the model's parameters and the learning rate
        self.parameters = parameters
        self.lr = lr
        self.model = model
    def zero_grad(self):
    # Reset the gradients of all parameters to zero
        self.model.weightsprime = [0] * len(self.model.weights)
        self.model.biasprime = 0
    def step(self):
    # Update
        self.model.update_weights_and_bias(self.lr)
        pass

<div dir="rtl">
این تابع ها برای محاسبه میانگین وزن ها و بایاس هاست که از مشتق یک بچ میانگین میگیریم و به اندازه میانگین آن 
update
می کنیم.

</div>

In [6]:

def mean_w(weights):
    sum_weights = weights[0].copy()
    length = len(weights) 


    for weight in range(1,len(weights)):
        for lw in range(len(weights[weight])):
            for nw in range(len(weights[weight][lw])):
                for w in range(len(weights[weight][lw][nw])):
                    sum_weights[lw][nw][w]  = sum_weights[lw][nw][w] + weights[weight][lw][nw][w]
    

   
    for lw in range(len(sum_weights)):
        for nw in range(len(sum_weights[lw])):
            for w in range(len(sum_weights[lw][nw])):
                sum_weights[lw][nw][w] /= length
    return sum_weights


def mean_b(weights):
    sum_weights = weights[0].copy()
    length = len(weights)   
    for weight in range(1,len(weights)):
        for lw in range(len(weights[weight])):
                for w in range(len(weights[weight][lw])):
                    sum_weights[lw][w]  += (weights[weight][lw][w])

   
    for lw in range(len(sum_weights)):
            for w in range(len(sum_weights[lw])):
                sum_weights[lw][w] /= length
    return sum_weights


w = [[[[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]], [[0.7, 0.8, 0.9], [1.0, 1.1, 1.2]]],[[[0.2, 0.2, 0.3], [0.4, 0.5, 0.6]], [[0.7, 0.8, 0.9], [1.0, 1.1, 1.2]]]]
mean_w(w)

[[[0.15000000000000002, 0.2, 0.3], [0.4, 0.5, 0.6]],
 [[0.7, 0.8, 0.9], [1.0, 1.1, 1.2]]]

<div dir="rtl">
در ابتدا یک دیتا ست میسازیم با استفاده از تابع 
make_blolbs
که در دو کلاس است و 1000 
نمونه دارد
و سپس 
model
را می سازیمکه دولایه است با 3و2 نورون
و ساز input
که تعداد 
فیچر هاست همان دو است در این مثال
و ابجکت
optimizer
را میسازیم و 
model 
را به آن می دهیم 
و به ازای تعداد ایپاک ها که 1000 تاست فور میزنیم.

weights_primes
و
bias_primes
مشتق نسبت به وزن ها وبایاس هارا ذخیره می کنید که در نهیات میانگین بگیریم و در ادامه فور مینیم که به ازای یک ایپاک
کل داده هارا ببینیم و مشتق نسبت وزن ها و بایاس های هر داده را ذخیره می کنیم 
ایتدا 
ورودی را به مذل می دهیم که همان 
x
هر داده است
و
y_hat
را که فوروارد یا کال کردن مدل است 
بدست می اورد
و با 
Backward
مشتق های پارامتر هارا بدست می اورین و به لست تمام مشتق ها و بایاس ها اضافه می کنیم.
و در انتهای این 
for
لاس برای این داده را محاسبه میکنمو با لاس های داده های قبلی در این بچ جمع می کنیم.
سپس از مشتق های پارا متر ها با 
mean_weights_prime
و
mean_bias_prime
میانگین می گیریم و با استفاده از این میانگین ها و
step
مدل را اپدیت می کنیم.
که در نهایت اپذیت می کنیم همه پرا متر هارا و 
cost
که میانگین لاست هارا با تقسیم مجموع به تعداد حساب میکنیم.
و پرینت می کنیم که پیشرفت را ببینیم و در نهایت 
و در نهایت 
acuracy 
را با توجه به این که چند تا درست داده است و چند درصد درست داده چاپ می کنیم.
و میبینم که 
cost 
کم میشود
و
accuracy 
هم زیاد می شود.

</div>

In [7]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=1000, n_features=2, centers=2, random_state=23)
# make a model

model = Mlp([5],2,['sigmoid','sigmoid'])
optim = Optimizer(1,model)
n_epochs = 1000
for _ in range(n_epochs):
    weights_primes = []
    bias_primes = []
    loss = 0
    for j in range(len(X)):
        model.set_input(X[j])
        y_hats = model()
        model.Backward(y[j])
        weights_prime = model.getweights_prime()
        bias_prime = model.getbias_prime()
        weights_primes.append(weights_prime)
        bias_primes.append(bias_prime)
        loss += model.loss(y[j])
    # without np
    mean_weights_prime  = mean_w(weights_primes)
    mean_bias_primes = mean_b(bias_primes)
    model.set_weights_prime(mean_weights_prime)
    model.set_bias_prime(mean_bias_primes)
    model.update_weights_and_bias(0.5)
    cost = loss/len(X)
    print(cost)






accuracy = 0
for i in range(len(X)):
    model.set_input(X[i])
    print(model.forward())
    if model.forward() > 0.5:
        if y[i] == 1:
            accuracy += 1
    else:
        if y[i] == 0:
            accuracy += 1
print(accuracy/len(X))


0.41037021630284454
0.37911840059030044
0.32755276385043375
0.2686062110727972
0.2210400752494141
0.18266042774777036
0.15329533761524353
0.1319503833913877
0.11651394817225694
0.10496873710428842
0.09589746396776695
0.0884298153509802
0.08205626142283255
0.0764778812921298
0.07151337073875698
0.0670467672551857
0.06299907327061642
0.059312930290179394
0.055944241851140025
0.052857488597552515
0.05002302749853243
0.04741547832581982
0.045012720307834125
0.042795240015160524
0.040745686579367835
0.03884855239978999
0.037089931850368074
0.03545733001416526
0.03393950479220757
0.03252633238835286
0.031208690106515034
0.029978352718561644
0.02882790002117737
0.027750633987894704
0.02674050437420191
0.025792041891650914
0.024900298216380127
0.024060792188399904
0.023269461618849695
0.022522620168834644
0.021816918803455054
0.021149311362042608
0.020517023821896838
0.0199175268682868
0.01934851141795368
0.01880786677645067
0.018293661141040356
0.017804124190247107
0.01733763152837447
0.01689